# Rens Flora Danica-datasættet  
Flora Danica-datasættet indeholder metadata i en xlsx-fil og mere end 3000 tiff-filer. I notebooken renser og udforsker vi de tmetadatasæt, som er tilgængeligt i LOAR.

## Importer bibliotekerne

In [2]:
import pandas as pd
import re
import requests

## Gør metadataet "tidy"

Metadataet kommer fra en datadump fra biblioteket.

Selvom datasættet umiddelbart ser ud til at være velstruktureret, kan vi få mere ud af det, hvis vi rydder lidt op i datae. 

Lad os tage et kig på data.

In [3]:
# Indlæs filen med Flora Danica metadata
print ('Indlæs data')
df = pd.read_excel(r'mekuni_flora_danica_data/Index_FloraDanica.xlsx')
print (f'Færdig. Dataframens form: {df.shape}')
print ('Inspicer de første tre rækker data.')

df.head(2)

Indlæs data
Færdig. Dataframens form: (3240, 10)
Inspicer de første tre rækker data.


,Record Name,Titel,Opstilling,Lokalitet,Ophav,År,Note,Taxonomisk gruppe,Hæfte,Copyright
0,floradanica_0001.tif,Rubus Chamaemorus Linn.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 1\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,Materialet er fri af ophavsret
1,floradanica_0002.tif,Pedicularis lapponica L.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 2\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,Materialet er fri af ophavsret


## Omdøb kolonner

Navnene på kolonnerne er rodet. Vi har en blanding af engelsk og dansk. For at omdøbe kolonnerne i dataframen, mapper vi de gamle navne til nogle nye navne og omdøber derefter kolonnerne.

In [3]:
# Define the mapping from old column names to new column names
column_rename_mapping = {
    "Record Name": "record_name",
    "Titel": "title",
    "Opstilling": "placement",
    "Lokalitet": "location",
    "Ophav": "author",
    "År": "year",
    "Note": "note",
    "Taxonomisk gruppe": "taxonomic_group",
    "Hæfte": "issue",
    "Copyright": "copyright"
}

# Rename the columns using the mapping
df.rename(columns=column_rename_mapping, inplace=True)

## Tabel nummer: tilføj information om tabelnummerne

Hvert billede i Flora Danica er markeret med et tabelnummer. Det er ofte øverst på billedet. Der er 3240 forskellige numre. Vi kan oprette en kolonne med tabelnumrene ved at tage indeksnumrene, der starter med 0, og tilføje én.

In [38]:
# modificer dataframen og tilføj kolonnen kaldet table_no  
print('modificer dataframen og tilføj kolonnen kaldet table_no')  
df = df.reset_index().rename(columns={'index': 'table_no'})  
df['table_no'] = df['table_no'] + 1  
df.head(2)

modificer dataframen og tilføj kolonnen kaldet table_no


,table_no,table_no,record_name,title,placement,location,author,year,note,taxonomic_group,issue,copyright
0,1,2,floradanica_0001.tif,Rubus Chamaemorus Linn.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 1\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,Materialet er fri af ophavsret
1,2,3,floradanica_0002.tif,Pedicularis lapponica L.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 2\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,Materialet er fri af ophavsret


## Kolonne: Ophav
Alle rækker i kolonnen "Ophav" indeholder den samme, identiske information, som er navnene og fødsels- og dødsdatoerne på flere af de botanikere, der har været ansvarlige for udgivelsesarbejdet.

Den ensartede information i ser således ud:

_'Hornemann, Jens Wilken (6.3.1770-30.7.1841) botaniker \nLange, Johan Martin Christian (20.3.1818-3.4.1898) botaniker \nLiebmann, Frederik Michael (10.10.1813-29.10.1856) botaniker \nMüller, Otto Frederik (2.3.1730-26.12.1784) botaniker \nOeder, Georg Christian (3.2.1728-28.1.1791) botaniker \nVahl, Martin (7.10.1749-24.12.1804) botaniker'_  

For at kunne sortere og filtrere datasættet baseret på oplysninger om ophav, har vi brug for detaljerede oplysninger om, hvilken forfatter der har været ansvarlig for udgivelsen af hvert billede. _[En sådan detaljeret oversigt kan findes på denne side](http://wayback-01.kb.dk/wayback/20101110095834/http://www2.kb.dk/udstillinger/floradanica/floradanica/editorer/oversigt.html)_




| Hæfte     | Tavle nr.   | År   | Udgiver                                   |
|-----------|-------------|------|-------------------------------------------|
| 1         | 1-60        | 1761 | G. C. Oeder                               |
| 2         | 61-120      | 1763 | G. C. Oeder                               |
| 3         | 121-180     | 1764 | G. C. Oeder                               |
| 4         | 181-240     | 1765 | G. C. Oeder                               |
| 5         | 241-300     | 1766 | G. C. Oeder                               |
| 6         | 301-360     | 1767 | G. C. Oeder                               |
| 7         | 361-420     | 1768 | G. C. Oeder                               |
| 8         | 421-480     | 1769 | G. C. Oeder                               |
| 9         | 481-540     | 1770 | G. C. Oeder                               |
| 10        | 541-600     | 1771 | G. C. Oeder                               |
| 11        | 601-660     | 1775 | O. F. Müller                              |
| 12        | 661-720     | 1777 | O. F. Müller                              |
| 13        | 721-780     | 1778 | O. F. Müller                              |
| 14        | 781-840     | 1780 | O. F. Müller                              |
| 15        | 841-900     | 1782 | O. F. Müller                              |
| 16        | 901-960     | 1787 | M. Vahl                                   |
| 17        | 961-1020    | 1790 | M. Vahl                                   |
| 18        | 1021-1080   | 1792 | M. Vahl                                   |
| 19        | 1081-1140   | 1794 | M. Vahl                                   |
| 20        | 1141-1200   | 1797 | M. Vahl                                   |
| 21        | 1201-1260   | 1799 | M. Vahl                                   |
| 22        | 1261-1320   | 1806 | J. W. Hornemann                           |
| 23        | 1321-1380   | 1808 | J. W. Hornemann                           |
| 24        | 1381-1440   | 1810 | J. W. Hornemann                           |
| 25        | 1441-1500   | 1813 | J. W. Hornemann                           |
| 26        | 1501-1560   | 1816 | J. W. Hornemann                           |
| 27        | 1561-1620   | 1818 | J. W. Hornemann                           |
| 28        | 1621-1680   | 1819 | J. W. Hornemann                           |
| 29        | 1681-1740   | 1821 | J. W. Hornemann                           |
| 30        | 1741-1800   | 1823 | J. W. Hornemann                           |
| 31        | 1801-1860   | 1825 | J. W. Hornemann                           |
| 32        | 1861-1920   | 1827 | J. W. Hornemann                           |
| 33        | 1921-1980   | 1829 | J. W. Hornemann                           |
| 34        | 1981-2040   | 1830 | J. W. Hornemann                           |
| 35        | 2041-2100   | 1832 | J. W. Hornemann                           |
| 36        | 2101-2160   | 1834 | J. W. Hornemann                           |
| 37        | 2161-2220   | 1836 | J. W. Hornemann                           |
| 38        | 2221-2280   | 1839 | J. W. Hornemann                           |
| 39        | 2281-2340   | 1840 | J. W. Hornemann                           |
| 40        | 2341-2400   | 1843 | S. Drejer, J. F. Schouw & J. Vahl         |
| 41        | 2401-2460   | 1845 | F. Liebmann                               |
| 42        | 2461-2520   | 1849 | F. Liebmann                               |
| 43        | 2521-2580   | 1852 | F. Liebmann                               |
| 44        | 2581-2640   | 1858 | Japetus Steenstrup & Johan Lange          |
| 45        | 2641-2700   | 1861 | Johan Lange                               |
| 46        | 2701-2760   | 1867 | Johan Lange                               |
| 47        | 2761-2820   | 1869 | Johan Lange                               |
| 48        | 2821-2880   | 1871 | Johan Lange                               |
| 49        | 2881-2940   | 1877 | Johan Lange                               |
| 50        | 2940-3000   | 1880 | Johan Lange                               |
| 51        | 3001-3060   | 1883 | Johan Lange                               |
| Suppl 1   | 1-60        | 1853 | F. Liebmann                               |
| Suppl 2   | 61-120      | 1865 | Johan Lange                               |
| Suppl 3   | 121-180     | 1874 | Johan Lange                               |



Lad os tilføje oplysningerne om tabeller og forfattere til hver række i datasættet. 

Bemærk, at de sidste tre linjer beskriver de 180 tabeller i tillægssiderne til Flora Danica.

In [37]:
#Opret en ny dataframe med detaljeret information om, hvordan tabelnumre relaterer sig til forfatterudskrifter.


# Rå data string
data = """
table_no\tyear\tauthor
1-60 	1761 	G. C. Oeder
61-120 	1763 	G. C. Oeder
121-180 	1764 	G. C. Oeder
181-240 	1765 	G. C. Oeder
241-300 	1766 	G. C. Oeder
301-360 	1767 	G. C. Oeder
361-420 	1768 	G. C. Oeder
421-480 	1769 	G. C. Oeder
481-540 	1770 	G. C. Oeder
541-600 	1771 	G. C. Oeder
601-660 	1775 	O. F. Müller
661-720 	1777 	O. F. Müller
721-780 	1778 	O. F. Müller
781-840 	1780 	O. F. Müller
841-900 	1782 	O. F. Müller
901-960 	1787 	M. Vahl
961-1020 	1790 	M. Vahl
1021-1080 	1792 	M. Vahl
1081-1140 	1794 	M. Vahl
1141-1200 	1797 	M. Vahl
1201-1260 	1799 	M. Vahl
1261-1320 	1806 	J. W. Hornemann
1321-1380 	1808 	J. W. Hornemann
1381-1440 	1810 	J. W. Hornemann
1441-1500 	1813 	J. W. Hornemann
1501-1560 	1816 	J. W. Hornemann
1561-1620 	1818 	J. W. Hornemann
1621-1680 	1819 	J. W. Hornemann
1681-1740 	1821 	J. W. Hornemann
1741-1800 	1823 	J. W. Hornemann
1801-1860 	1825 	J. W. Hornemann
1861-1920 	1827 	J. W. Hornemann
1921-1980 	1829 	J. W. Hornemann
1981-2040 	1830 	J. W. Hornemann
2041-2100 	1832 	J. W. Hornemann
2101-2160 	1834 	J. W. Hornemann
2161-2220 	1836 	J. W. Hornemann
2221-2280 	1839 	J. W. Hornemann
2281-2340 	1840 	J. W. Hornemann
2341-2400 	1843 	S. Drejer, J. F. Schouw & J. Vahl
2401-2460 	1845 	F. Liebmann
2461-2520 	1849 	F. Liebmann
2521-2580 	1852 	F. Liebmann
2581-2640 	1858 	Japetus Steenstrup & Johan Lange
2641-2700 	1861 	Johan Lange
2701-2760 	1867 	Johan Lange
2761-2820 	1869 	Johan Lange
2821-2880 	1871 	Johan Lange
2881-2940 	1877 	Johan Lange
2940-3000 	1880 	Johan Lange
3001-3060 	1883 	Johan Lange
3060-3119 	1853 	F. Liebmann
3120-3179 	1865 	Johan Lange
3180-3240 	1874 	Johan Lange
"""

# Opdel data i linjer og del hver linje i kolonner
lines = data.strip().split('\n')
columns = lines[0].split('\t')  # Ekstraher data
data_rows = [re.split(r'\s{2,}', line.strip()) for line in lines[1:]]  # Opret dataframes med detaljeret info
detailed_info = pd.DataFrame(data_rows, columns=columns)

# Sørg for, at 'table_no' behandles som en streng
detailed_info['table_no'] = detailed_info['table_no'].astype(str)

# Liste til at gemme nye dataframes
new_dfs = []

# Gennemgå hver række i dataframen
for i, row in detailed_info.iterrows():
    # Parse intervallet
    start, end = map(int, row['table_no'].split('-'))
    
    # Opret indeksinterval
    indices = range(start, end + 1)
    
    # Opret et nyt dataframe
    new_df = pd.DataFrame({
        'table_no': indices,
        'author_st': [row['author']] * len(indices)
    })
    
    # Tilføj til listen
    new_dfs.append(new_df)

# Nu indeholder new_dfs alle de nye dataframes
# og de kan sammenføjes til ét stort dataframe
detailed_info_df = pd.concat(new_dfs).reset_index(drop=True)

print ('Nye data om "author" er tilføjet til kolonnen "author_st".\n')
# Kombiner datasæt
df_w_year_author = pd.merge(df, detailed_info_df, how='left', on='table_no')
df_w_year_author.head(2)

Nye data om "author" er tilføjet til kolonnen "author_st".



,table_no,record_name,title,placement,location,author,year,note,taxonomic_group,issue,copyright,author_st
0,1,floradanica_0001.tif,Rubus Chamaemorus Linn.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 1\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,Materialet er fri af ophavsret,G. C. Oeder
1,2,floradanica_0002.tif,Pedicularis lapponica L.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 2\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,Materialet er fri af ophavsret,G. C. Oeder


## Kolonne 'issue': Rens den og tilføj en ny kolonne kaldet 'issue_st'

Rækkerne i kolonnen 'issue' indeholder denne lange streng: _Digitale Samlinger: Billeder:Særudgivelser:Flora Danica:Hæfte:_ . 

Strengen er en sti, der angiver placeringen i de Digitale Samlinger. Vi har ikke brug for informationen om stien. De data, vi virkelig har brug for, er dem, der er tilbage, når vi fjerner den lange streng.

Vi kan få dem ved at rense strengen, og det gør vi ved at erstatte den lange streng med ingenting. Vi gør det ved at skrive en funktion kaldet _clean_issue_ og bruge den indbyggede Python-metode .replace(). Vi anvender derefter funktionen på kolonnen 'issue' og tilføjer dataene til dataframen i en ny kolonne kaldet _issue_st_.

In [10]:
print (df_w_year_author.at[0,'issue'])

long_string = df_w_year_author.at[0,'issue']

clean_string = long_string.replace('Digitale Samlinger: Billeder:Særudgivelser:Flora Danica:Hæfte:', '')
print(clean_string)

Digitale Samlinger: Billeder:Særudgivelser:Flora Danica:Hæfte:Hft.  1
Hft.  1


In [36]:
def clean_issue(text_string_in):
    text_string_out = text_string_in.replace('Digitale Samlinger: Billeder:Særudgivelser:Flora Danica:Hæfte:', '')
    return text_string_out


df_w_year_author['issue_st'] = df_w_year_author['issue'].apply( lambda x : clean_issue(x))
df_w_year_author.head(2)

,table_no,record_name,title,placement,location,author,year,note,taxonomic_group,issue,copyright,author_st,issue_st,taxonomic_group_st
0,1,floradanica_0001.tif,Rubus Chamaemorus Linn.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 1\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,free,G. C. Oeder,Hft. 1,Karplanter
1,2,floradanica_0002.tif,Pedicularis lapponica L.,Fol. Top. Bot. Danmark,Danmark\nNorge,"Hornemann, Jens Wilken (6.3.1770-30.7.1841) bo...",1761,"Flora Danica Hft. 1, Tab. 2\n\nFigur 1\nLatins...",Digitale Samlinger: Digitale Samlinger: Billed...,Digitale Samlinger: Billeder:Særudgivelser:Flo...,free,G. C. Oeder,Hft. 1,Karplanter


## Kolonnenavn: taxonomic_group  
Værdierne i kolonnen er lidt rodede.

De indeholder både oplysninger om navnet på samlingen og oplysninger om den taksonomiske gruppe, som planten på billedet tilhører.

In [12]:
df_w_year_author.at[0, 'taxonomic_group']

'Digitale Samlinger: Digitale Samlinger: Billeder:Særudgivelser:Flora Danica:Taxonomisk gruppe:Karplanter'

## Uddrag de relevante oplysninger og tilføj dem til en ny kolonne

Den egentlige information om taksonomi ville være "Karplanter", mens resten af tekststrengen kan betragtes som støj.

Lad os prøve at udtrække de relevante data og tilføje dem til en ny kolonne.

In [13]:
# Tilgå en enkelt værdi
S = df_w_year_author.at[0, 'taxonomic_group']
# Split på ':' og tag listens sidste element (den information som vi egentlig ønsker)
group_val = S.split(':')[-1]
group_val

'Karplanter'

Nu skriver vi en funktion og benytter den til at dataene og tilføje dem til en ny kolonne. Når vi inspicerer datasættet, ser det umiddelbart ud til, at de fleste planter tilhører taksonomigruppen "Karplanter".

In [14]:
def get_taxonomy_data(S):
    group_val = S.split(':')[-1]
    return group_val


df_w_year_author['taxonomic_group_st'] = df_w_year_author['taxonomic_group'].apply(lambda x:get_taxonomy_data(x))
# Inspiser data i en den ny kolonne
print (df_w_year_author['taxonomic_group_st'].value_counts())

taxonomic_group_st
Karplanter           2073
Svampe                391
Mosser                331
Alger                 228
Laver                 163
Slimsvampe             39
Ukendt                 15
Taxonomisk gruppe       1
Lave                    1
Name: count, dtype: int64


## Kolonne copyright: modificer tekststreng

Værdierne i kolonnen "copyright" alle sammen de samme. De består af en tekststreng, hvor der står "Materialet er fri af ophavsret".

Lad os ændrer teksten til det kortere "fri".

In [15]:
print (df_w_year_author.at[0, 'copyright'])

text_string = df_w_year_author.at[0, 'copyright']
new_text_string = text_string.replace('Materialet er fri af ophavsret', 'fri')
print (new_text_string)

Materialet er fri af ophavsret
fri


In [16]:
def clean_copyright(S):
    new_text_string  = S.replace('Materialet er fri af ophavsret', 'free')
    return new_text_string


df_w_year_author['copyright'] = df_w_year_author['copyright'].apply(lambda x : clean_copyright(x))

## Kolonnenavn: note

Værdierne i datasættets andre kolonner har også flere værdier i andre kolonner. For eksempel i kolonnen 'note'. Lad os se nærmere på værdierne i den første række af denne kolonne.

In [17]:
print (f'\nNote:\n{df_w_year_author.at[0,"note"]}\n\n')


Note:
Flora Danica Hft. 1, Tab. 1

Figur 1
Latinsk navn: Rubus chamaemorus L.
Dansk slægtsnavn: Multebær
Dansk familienavn: Rosenfamilien
Latinsk familienavn: Rosaceae




Nedenfor bliver der læst data fra 'Note' kolonnen, og ved hjælp af regulære udtryk bliver der søgt efter specifikke oplysninger, nemlig det latinske familienavn, det latinske navn og lange nomenklatur. Funktionen `parse_notes` udtrækker dataen, og resultatet tilføjes som nye kolonner til den oprindelige DataFrame. 


In [37]:
# Funktion der finder de relevant data fra værdierne i kolonnen 'note' 
print ('Find relevant data i værdierne i kolonnen "note"')
def parse_notes(note):
    # Start med default værdier
    latinsk_familienavn = None
    latinsk_navn = None
    lange_nomenklator = None

    # Regular udtryk, der finder relevant information
    latinsk_familienavn_pattern = r'Latinsk familienavn:\s*(.*)'
    latinsk_navn_pattern = r'Latinsk navn:\s*(.*)'
    lange_nomenklator_pattern = r'Lange nomenklator:\s*(.*)'

    # Søg efter relevant information
    latinsk_familienavn_match = re.search(latinsk_familienavn_pattern, note)
    if latinsk_familienavn_match:
        latinsk_familienavn = latinsk_familienavn_match.group(1).strip()

    latinsk_navn_match = re.search(latinsk_navn_pattern, note)
    if latinsk_navn_match:
        latinsk_navn = latinsk_navn_match.group(1).strip()

    lange_nomenklator_match = re.search(lange_nomenklator_pattern, note)
    if lange_nomenklator_match:
        lange_nomenklator = lange_nomenklator_match.group(1).strip()

    # Kombiner Latinsk navn and Lange nomenklator
    combined_latinsk_navn = latinsk_navn if latinsk_navn != '-' else lange_nomenklator

    return latinsk_familienavn, combined_latinsk_navn

# Applicer funktionen til dataframen
parsed_data = df['note'].apply(parse_notes)
df_parsed = pd.DataFrame(parsed_data.tolist(), columns=['latin_family_name', 'latin_name'])
print ('Færdig')

# Saml data i "Note" kolonnen til en dataframe
print ('Sammenkæd dataen med den oprindelige dataframe')
concat_df = pd.concat([df_w_year_author,df_parsed], axis=1)
print (f'Færdig. Dataframens form: {concat_df.shape}')

Find relevant data i værdierne i kolonnen "note"
Færdig
Sammenkæd dataen med den oprindelige dataframe
Færdig. Dataframens form: (3242, 16)


## Opret et "del-datasæt" og gem det som en CSV fil
 
Datasættet er renere og mere velorganiseret end før. 

Det er lettere for os at bruge det til analyse og visualiseringer. 

Dog er det kun nogle af kolonnerne, vi behøver i det videre arbejde. Derfor udvælger jeg nogle af kolonnerne til mit "del-datasæt", som jeg gemmer, som en CSV-fil.

In [24]:
subset_df = concat_df[['table_no', 'record_name', 'title', 'year', 'author_st', 'taxonomic_group_st', 'issue', 'latin_family_name', 'latin_name', 'copyright']]
subset_df.to_csv(r'.\mekuni_flora_danica_data\flora_danica_tidy_format.csv', index=False)